<a href="https://colab.research.google.com/github/apu-eee-sec/Multiplier-VHDL/blob/main/Python%20test%20with%20position%20indicate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
from PIL import Image, ImageDraw, ImageFont

def encode_3bit(segment):
    encoding = {
        "000": 0, "001": +1, "010": +1, "011": +2,
        "100": -2, "101": -1, "110": -1, "111": 0
    }
    return encoding.get(segment, 0)

def encode_8bit(B):
    """Performs Booth encoding on an 8-bit binary string B."""
    if len(B) != 8 or not set(B).issubset({'0', '1'}):
        return "Invalid input! Must be an 8-bit binary string."

    encoded_values = [
        encode_3bit(B[0:3]),
        encode_3bit(B[2:5]),
        encode_3bit(B[4:7]),
        encode_3bit(B[6:] + "0")
    ]
    return encoded_values

def twos_complement(binary_str):
    """Compute 2's complement of an 8-bit binary string."""
    inverted_bits = ''.join('1' if bit == '0' else '0' for bit in binary_str)
    twos_comp_int = int(inverted_bits, 2) + 1
    twos_comp_bin = bin(twos_comp_int)[2:].zfill(8)
    return twos_comp_bin[-8:]

def get_partial_product(A, encoded_B):
    """Compute partial products based on A and encoded values of B."""
    if len(A) != 8 or not set(A).issubset({'0', '1'}):
        return "Invalid input! A must be an 8-bit binary string."
    msb=[0,0,0,0]
    partial_products = []

    for i in range(len(encoded_B)):
        if encoded_B[i] == 0:
            partial_products.append("00000000")
            msb[3-i]="0"
        elif encoded_B[i] == 1:
            partial_products.append(A)
            msb[3-i]=str(int(A[0]) ^ 0)

        elif encoded_B[i] == 2:
            partial_products.append(A + "0")
            msb[3-i]=str(int(A[0]) ^ 0)
        elif encoded_B[i] == -1:
            partial_products.append(twos_complement(A))
            msb[3-i]=str(int(A[0]) ^ 1)
        elif encoded_B[i] == -2:
            partial_products.append(twos_complement(A) + "0")
            msb[3-i]=str(int(A[0]) ^ 1)
            #print("one:",i)

    partial_products = partial_products[::-1]
    #print(msb[1])
    for i in range(len(partial_products)):
        msb2 = partial_products[i][0]

        if i == 0:
            target_length = 16
        elif i == 1:
            target_length = 14
        elif i == 2:
            target_length = 12
        elif i == 3:
            target_length = 10

        current_length = len(partial_products[i])
        bits_to_add = target_length - current_length
        fill_bits = msb[i] * bits_to_add
        partial_products[i] = fill_bits + partial_products[i]

    return partial_products

def save_partial_products_with_encode_b_and_A(partial_products, encoded_B, A, new_partial1, new_partial2, filename="partial_products_image_with_encode_b_and_A.png"):
    """Save partial products with encoded B and A values as an image, including new partial products."""
    # Define image size based on the number of lines and length of strings
    image_width = 540  # Increased width for more space between columns
    image_height = 80 * (len(partial_products) + 3)  # Increased height for line spacing (allowing space for the header, separator, and new partial products)
    background_color = (255, 255, 255)  # White background
    text_color = (0, 0, 0)  # Black text

    # Create a blank white image
    img = Image.new("RGB", (image_width, image_height), background_color)
    draw = ImageDraw.Draw(img)

    # Set font size to a larger value and ensure we use a scalable font
    try:
        # Try to load a scalable font (e.g., Arial) with a larger font size (e.g., 40)
        font = ImageFont.truetype("arial.ttf", 40)
        position= ImageFont.truetype("arial.ttf", 20)
    except IOError:
        # If the font is not available, use a basic default font but with larger size
        font = ImageFont.load_default()
        position = ImageFont.load_default()

    # Set the gap between characters in each partial product (increased for readability)
    column_gap = 30
    line_height = 50  # Space between lines

    # Draw bit positions
    max_len = max(len(p) for p in partial_products)
    for i in range(16):
        # Ensure LSB alignment with partial product LSB and Encode B
        x_pos = (max_len - len("+")) * column_gap - 195  # Adjust to align LSB of A with partial product
        draw.text((38 + 0.985*i * 31, 0), str(abs(i-15)), fill=(128,128,128), font=position)

    # Draw A values on top of the partial products and Encode B
    max_len = max(len(p) for p in partial_products)
    for i, value in enumerate(A):
        # Ensure LSB alignment with partial product LSB and Encode B
        x_pos = (max_len - len(str(value))) * column_gap - 170  # Adjust to align LSB of A with partial product
        draw.text((x_pos + i * column_gap, 20), str(value), fill=text_color, font=font)

    # Draw Encode B values on top of the partial products
    for i, value in enumerate(encoded_B):
        # Ensure LSB alignment with partial product LSB
        # Determine the X position based on the length of the partial product
        if value>=0:
            x_pos = (max_len - 3) * column_gap+10  # Adjust to align LSB of Encode B
        else:
            x_pos = (max_len - 3.35) * column_gap+10  # Adjust to align LSB of Encode B
        draw.text((x_pos+i*column_gap, line_height+8), str(value), fill=text_color, font=font)

        #print(value)
    # Draw separator line between Encode B and partial products
    draw.line([(35, line_height * 2), (image_width, line_height * 2)], fill=text_color, width=2)

    # Draw partial products with larger space between columns
    for index, product in enumerate(partial_products):
        # Draw each character in the partial product with more space between columns
        for i, char in enumerate(product):
            if i==0:
                draw.text((10 + i * column_gap, (index + 2.25) * line_height), chr(65+index), fill=(128,128,128), font=position)
            draw.text((40 + i * column_gap, (index + 2) * line_height), char, fill=text_color, font=font)

        # After the third partial product, add a separator line and the new partial products
        if index == 3:
            y_position = (index + 3) * line_height
            # Draw separator line
            draw.line([(35, y_position), (image_width, y_position)], fill=text_color, width=2)
            y_position += line_height

            # Draw new_partial1
            for i, char in enumerate(new_partial1):
                if i==0:
                    draw.text((10 + i * column_gap, (index + 3.25) * line_height), chr(69), fill=(128,128,128), font=position)
                draw.text((40 + i * column_gap,  (index + 3) * line_height), char, fill=text_color, font=font)


            # Draw new_partial2
            for i, char in enumerate(new_partial2):
                if i==0:
                    draw.text((10 + i * column_gap, (index + 4.25) * line_height), chr(70), fill=(128,128,128), font=position)
                draw.text((40 + i * column_gap, (index + 4) * line_height), char, fill=text_color, font=font)

            # Draw another separator line after new partial products
            y_position += line_height
            draw.line([(35, y_position), (image_width, y_position)], fill=text_color, width=2)

             # Draw final output binary
            for i, char in enumerate(output):
                draw.text((40 + i * column_gap, (index + 5) * line_height), char, fill=text_color, font=font)

            # Draw final output decimal
            for i, char in enumerate(str(decimal_value)):
                draw.text((40 + i * column_gap, (index + 7) * line_height), char, fill=text_color, font=font)

    # Save the image
    img.save(filename)



def sum_selected_bits(partial1, partial2, start_index=4, get_carry=1):
    """
    Sum bits from the specified start_index to the MSB with carry propagation
    but only between the given two partial products.
    """
    max_length = max(len(partial1), len(partial2))
    p1 = partial1.rjust(max_length, partial1[0])  # Sign-extend
    p2 = partial2.rjust(max_length-2, partial2[0])  # Sign-extend
    #carry = 0
    if get_carry==1:
        carry=int(partial_products[0][5]) & int(partial_products[1][5]) & int(partial_products[2][5])
    elif get_carry==2:
        carry=int(partial_products[1][5]) & int(partial_products[2][5]) & int(partial_products[3][5])
    else:
        carry=0
    result = []
    #print("carry", carry);

    for i in range(start_index, 0, -1):
        column_sum = int(carry) + int(p1[i]) + int(p2[i])

        s = column_sum % 2  # Sum bit
        carry = column_sum // 2  # Carry bit
        result.append(str(s))

    result.append('*')
    result.reverse()

    return ''.join(result)


def sum_selected_bits_apprx_4_2(partial1, partial2,partial3,partial4, start_index=9, sum=1):
    """
    Sum bits from the specified start_index to the MSB with carry propagation
    but only between the given two partial products.
    """
    max_length = max(len(partial1), len(partial2))
    p1 = partial1.rjust(max_length, partial1[0])  # Sign-extend
    p2 = partial2.rjust(max_length-2, partial2[0])  # Sign-extend
    p3 = partial3.rjust(max_length-4, partial3[0])  # Sign-extend
    p4 = partial4.rjust(max_length-6, partial4[0])  # Sign-extend    carry = 0
    result = []
    carry= 0
    sum1=0
    for i in range(start_index, 4, -1):
        if sum == 1:
            sum1 = int(p1[i]) and int(p2[i]) or int(p3[i]) or int(p4[i])
        elif sum == 2:
            sum1 = int(p1[i]) or int(p2[i]) or int(p3[i]) and int(p4[i])
        result.append(str(sum1))
    if sum == 1:
        result.append(str(new_partial1))
    elif sum == 2:
        result.append(str(new_partial2))
    result.reverse()

    return ''.join(result)

def sum_selected_bits_apprx_3_2(partial1, partial2,partial3, start_index=9, sum=1):
    """
    Sum bits from the specified start_index to the MSB with carry propagation
    but only between the given two partial products.
    """
    max_length = max(len(partial1), len(partial2))
    p1 = partial1.rjust(max_length, partial1[0])  # Sign-extend
    p2 = partial2.rjust(max_length-2, partial2[0])  # Sign-extend
    p3 = partial3.rjust(max_length-4, partial3[0])  # Sign-extend
    result = []
    carry= 0
    for i in range(start_index, 9, -1):
        if sum == 1:
           sum1 = int(p1[i]) and int(p2[i]) or int(p3[i])
        elif sum == 2:
           sum1 = int(p1[i]) or int(p2[i])
        result.append(str(sum1))
    if sum == 1:
        result.append(str(new_partial1))
    elif sum == 2:
        result.append(str(new_partial2))
    result.reverse()

    return ''.join(result)


def copy_selected_bits(partial1, partial2, start_index=15,copy=1):
    """
    Sum bits from the specified start_index to the MSB with carry propagation
    but only between the given two partial products.
    """
    max_length = max(len(partial1), len(partial2))
    p1 = partial1.rjust(max_length, partial1[0])  # Sign-extend
    p2 = partial2.rjust(max_length-2, partial2[0])  # Sign-extend
    result = []
    #print(p1)
    #print(p2)
    #print(p3)
    #print(p4)
    for i in range(start_index, 11, -1):
        if copy == 1:
           copy1 = int(p1[i])
        elif copy == 2:
           copy1 = int(p2[i])
        result.append(str(copy1))
    if copy == 1:
        result.append(str(new_partial1))
    elif copy == 2:
        result.append(str(new_partial2))

    result.reverse()

    return ''.join(result)


def final_output(partial1, partial2):
    """
    Sum bits from the specified start_index to the MSB with carry propagation
    but only between the given two partial products.
    """
    max_length = max(len(partial1), len(partial2))
    p1 = partial1.rjust(max_length, partial1[0])  # Sign-extend
    result = []
    # Perform XOR operation to get signed bit
    xor_result = str(int(A[0]) ^ int(B[0]))
    final_out=sum_selected_bits(new_partial1, new_partial2, start_index=13, get_carry=0)
    final_out=final_out.replace("*", xor_result, 1)
    #print(p2)
    #print(p3)
    #print(p4)
    for i in range(15, 13, -1):
        copy1 = int(p1[i])
        result.append(str(copy1))

    result.reverse()

    final_out=final_out+''.join(result)
    return final_out

def binary_to_signed_decimal(binary_str):
    # Convert binary string to integer
    num = int(binary_str, 2)
    # Check if the number is negative (if the most significant bit is 1)
    if num >= 2**15:
        num -= 2**16  # Convert to signed 16-bit integer
    return num

def decimal_to_8bit_binary(num):
    if num < 0:
        num = (1 << 8) + num  # Convert negative number using two's complement
    return format(num & 0xFF, '08b')  # Ensure 8-bit format


# Example usage
DEC_A=-128
DEC_B=7
A = decimal_to_8bit_binary(DEC_A) # Example A
B = decimal_to_8bit_binary(DEC_B)  # Example B

encoded_B = encode_8bit(B)  # Get encoded B values
print("Encoded B:", encoded_B)  # Expected: [+1, +2, -1, -2]

partial_products = get_partial_product(A, encoded_B)
print("Reversed Partial Products with MSB filling:", partial_products)

# Compute the correct sums
new_partial1 = sum_selected_bits(partial_products[0], partial_products[1], start_index=4,get_carry=1)
new_partial1 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=1)
new_partial1 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=1)
new_partial1 = copy_selected_bits(partial_products[0], partial_products[1], start_index=15,copy=1)


new_partial2 = sum_selected_bits(partial_products[2], partial_products[3], start_index=4, get_carry=2)
new_partial2 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=2)
new_partial2 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=2)
new_partial2 = copy_selected_bits(partial_products[0], partial_products[1], start_index=13,copy=2)

#determine final output
output=final_output(partial_products[0], partial_products[1])

decimal_value = str(DEC_A)+" x "+str(DEC_B)+" = "+str(binary_to_signed_decimal(output))
print("output decimal ", decimal_value)
print("output binary ", output)

# Save the partial products and encoded B and A as an image, including new partial products
save_partial_products_with_encode_b_and_A(partial_products, encoded_B, A, new_partial1, new_partial2, str(DEC_A)+"x"+str(DEC_B)+".png")

Encoded B: [0, 0, 2, -1]
Reversed Partial Products with MSB filling: ['0000000010000000', '11111100000000', '000000000000', '0000000000']
output decimal  -128 x 7 = -896
output binary  1111110010000000


In [38]:
import csv

# Open a CSV file to write
with open("A x B.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["a", "b","Approximate binary","Approximate decimal","Exact result" ,"Exact-Approximate"])

    # Loop through values of a and b
    for a in range(128):
        for b in range(128):
            exact_result = a * b
            # Example usage
            A = decimal_to_8bit_binary(a) # Example A
            B = decimal_to_8bit_binary(b)  # Example B

            encoded_B = encode_8bit(B)  # Get encoded B values
            #print("Encoded B:", encoded_B)  # Expected: [+1, +2, -1, -2]

            partial_products = get_partial_product(A, encoded_B)
            #print("Reversed Partial Products with MSB filling:", partial_products)

            # Compute the correct sums
            new_partial1 = sum_selected_bits(partial_products[0], partial_products[1], start_index=4)
            new_partial1 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=1)
            new_partial1 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=1)
            new_partial1 = copy_selected_bits(partial_products[0], partial_products[1], start_index=15,copy=1)


            new_partial2 = sum_selected_bits(partial_products[2], partial_products[3], start_index=4)
            new_partial2 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=2)
            new_partial2 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=2)
            new_partial2 = copy_selected_bits(partial_products[0], partial_products[1], start_index=13,copy=2)

            #determine final output
            output=final_output(partial_products[0], partial_products[1])
            decimal_value = binary_to_signed_decimal(output)
            #print("output decimal ", decimal_value)
            #print("output binary ", output)

            writer.writerow([a, b,output,decimal_value,exact_result,int(exact_result-decimal_value)])
# Save the partial products and encoded B and A as an image, including new partial products
#save_partial_products_with_encode_b_and_A(partial_products, encoded_B, A, new_partial1, new_partial2, "partial_products_image_with_new_partials.png")

print("CSV file generated successfully.")

CSV file generated successfully.


In [40]:
import csv

# Open a CSV file to write
with open("-A x -B.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["a", "b","Approximate binary","Approximate decimal","Exact result" ,"Exact-Approximate"])

    # Loop through values of a and b
    for a in range(-128,1):
        for b in range(-128,1):
            exact_result = a * b
            # Example usage
            A = decimal_to_8bit_binary(a) # Example A
            B = decimal_to_8bit_binary(b)  # Example B

            encoded_B = encode_8bit(B)  # Get encoded B values
            #print("Encoded B:", encoded_B)  # Expected: [+1, +2, -1, -2]

            partial_products = get_partial_product(A, encoded_B)
            #print("Reversed Partial Products with MSB filling:", partial_products)

            # Compute the correct sums
            new_partial1 = sum_selected_bits(partial_products[0], partial_products[1], start_index=4)
            new_partial1 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=1)
            new_partial1 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=1)
            new_partial1 = copy_selected_bits(partial_products[0], partial_products[1], start_index=15,copy=1)


            new_partial2 = sum_selected_bits(partial_products[2], partial_products[3], start_index=4)
            new_partial2 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=2)
            new_partial2 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=2)
            new_partial2 = copy_selected_bits(partial_products[0], partial_products[1], start_index=13,copy=2)

            #determine final output
            output=final_output(partial_products[0], partial_products[1])
            decimal_value = binary_to_signed_decimal(output)
            #print("output decimal ", decimal_value)
            #print("output binary ", output)

            writer.writerow([a, b,output,decimal_value,exact_result,int(exact_result-decimal_value)])
# Save the partial products and encoded B and A as an image, including new partial products
#save_partial_products_with_encode_b_and_A(partial_products, encoded_B, A, new_partial1, new_partial2, "partial_products_image_with_new_partials.png")

print("CSV file generated successfully.")

CSV file generated successfully.


In [37]:
import csv

# Open a CSV file to write
with open("A x -B.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["a", "b","Approximate binary","Approximate decimal","Exact result" ,"Exact-Approximate"])

    # Loop through values of a and b
    for a in range(128):
        for b in range(-128,1):
            exact_result = a * b
            # Example usage
            A = decimal_to_8bit_binary(a) # Example A
            B = decimal_to_8bit_binary(b)  # Example B

            encoded_B = encode_8bit(B)  # Get encoded B values
            #print("Encoded B:", encoded_B)  # Expected: [+1, +2, -1, -2]

            partial_products = get_partial_product(A, encoded_B)
            #print("Reversed Partial Products with MSB filling:", partial_products)

            # Compute the correct sums
            new_partial1 = sum_selected_bits(partial_products[0], partial_products[1], start_index=4)
            new_partial1 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=1)
            new_partial1 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=1)
            new_partial1 = copy_selected_bits(partial_products[0], partial_products[1], start_index=15,copy=1)


            new_partial2 = sum_selected_bits(partial_products[2], partial_products[3], start_index=4)
            new_partial2 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=2)
            new_partial2 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=2)
            new_partial2 = copy_selected_bits(partial_products[0], partial_products[1], start_index=13,copy=2)

            #determine final output
            output=final_output(partial_products[0], partial_products[1])
            decimal_value = binary_to_signed_decimal(output)
            #print("output decimal ", decimal_value)
            #print("output binary ", output)

            writer.writerow([a, b,output,decimal_value,exact_result,int(exact_result-decimal_value)])
# Save the partial products and encoded B and A as an image, including new partial products
#save_partial_products_with_encode_b_and_A(partial_products, encoded_B, A, new_partial1, new_partial2, "partial_products_image_with_new_partials.png")

print("CSV file generated successfully.")

CSV file generated successfully.


In [36]:
import csv

# Open a CSV file to write
with open("-A x B.csv", "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["a", "b","Approximate binary","Approximate decimal","Exact result" ,"Exact-Approximate"])

    # Loop through values of a and b
    for a in range(-128,1):
        for b in range(128):
            exact_result = a * b
            # Example usage
            A = decimal_to_8bit_binary(a) # Example A
            B = decimal_to_8bit_binary(b)  # Example B

            encoded_B = encode_8bit(B)  # Get encoded B values
            #print("Encoded B:", encoded_B)  # Expected: [+1, +2, -1, -2]

            partial_products = get_partial_product(A, encoded_B)
            #print("Reversed Partial Products with MSB filling:", partial_products)

            # Compute the correct sums
            new_partial1 = sum_selected_bits(partial_products[0], partial_products[1], start_index=4)
            new_partial1 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=1)
            new_partial1 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=1)
            new_partial1 = copy_selected_bits(partial_products[0], partial_products[1], start_index=15,copy=1)


            new_partial2 = sum_selected_bits(partial_products[2], partial_products[3], start_index=4)
            new_partial2 = sum_selected_bits_apprx_4_2(partial_products[0], partial_products[1],partial_products[2],partial_products[3], start_index=9,sum=2)
            new_partial2 = sum_selected_bits_apprx_3_2(partial_products[0], partial_products[1],partial_products[2], start_index=11,sum=2)
            new_partial2 = copy_selected_bits(partial_products[0], partial_products[1], start_index=13,copy=2)

            #determine final output
            output=final_output(partial_products[0], partial_products[1])
            decimal_value = binary_to_signed_decimal(output)
            #print("output decimal ", decimal_value)
            #print("output binary ", output)

            writer.writerow([a, b,output,decimal_value,exact_result,int(exact_result-decimal_value)])
# Save the partial products and encoded B and A as an image, including new partial products
#save_partial_products_with_encode_b_and_A(partial_products, encoded_B, A, new_partial1, new_partial2, "partial_products_image_with_new_partials.png")

print("CSV file generated successfully.")

CSV file generated successfully.
